<img src="https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png" alt="logo_ironhack_blue 7" width="100" height="100">

# Ironhack Mid-Bootcamp Project

## Project Overview

Electric Vehicles (EVs) have been around for a couple decades, but its adoption has really picked up in the last 3 years. In this project we will analyze the increasing demand of EVs and who's leading it, figure out in which stage of adoption we are (according to Everett M. Rogger's [*Diffusion of Innovation Theory*](https://en.wikipedia.org/wiki/Diffusion_of_innovations)) and try to estimate how long will it take for the all car transportation to become electric by finding the EV adoption curve.

According to Rogers, when analyzing the adoption of an innovation, the population penetration follows a **normal distribution**, therefore the market share curve will follow its **cumulative distribution function**.
<div align="center">
<img src="http://steveboese.squarespace.com/storage/adoption_of_tech_no_title.jpg?__SQUARESPACE_CACHEVERSION=1450230149150" alt="Adoption curve examples" width="500">
<img src="https://i0.wp.com/ondigitalmarketing.com/wp-content/uploads/2012/01/640px-Diffusionofideas.png?ssl=1" alt="Adoption curve" width="500">
</div>



## Objective
The objective if this project is to answer the following questions:
1. What trend (linear, cuadratic, exponential...) is EV adoption currently following?
2. What is the current stage of Electric Vehicle (EV) adoption? Are we in the phase of Innovators, Early adopters, Early majority, Late majority, or Laggards?
4. Which countries are leading the adoption of EVs? And lagging in the adoption?
5. Which countries are adopting EV's faster (rate of adoption)?
6. How long will full EV adoption take?

Extra:
1. Which companies are leading the transition from ICE (Internal Combustion Engine) vehicles to EVs?
2. Compare the adoption of electric cars to other vehicles (trucks, vans and buses)

### Finding the EV adoption curve:
+ Assuming our datapoints follow a normal distribution, our sales share data points correspond to x values in the Cumulative Distribution Function:
    + $CDF = \Phi(x) = 0.5 * [1 + \frac{erf((x - \mu)}{(\sigma * \sqrt{2}))}]$ where μ is the mean, σ is the standard deviation, and erf() is the error function. We want to find μ and σ so that this equation holds for all of our points. This leads to a system of n nonlinear equations.
    + Using scipy's least squares solver we will solve μ & σ for the given dataset. With this curve we will be able to predict the rate of adoption of EVs worldwide.

## Datasets

+ 2010-2022 Global data on EV sales -> [Global EV Data Explorer](https://www.iea.org/data-and-statistics/data-tools/global-ev-data-explorer)
+ 2010-2022 Global data on car sales -> [Global Passenger Car Sales](https://www.iea.org/data-and-statistics/charts/passenger-car-sales-2010-2022)
+ Historical data on company sales and sales share:
    + [Best Selling Cars](https://www.best-selling-cars.com/brands/2021-full-year-global-volkswagen-brand-worldwide-car-sales-by-model-and-country/)
    + [Toyota](https://global.toyota/en/newsroom/corporate/20966057.html)
    + [EV Adoption](https://evadoption.com/ev-sales/evs-percent-of-vehicle-sales-by-brand/)

# 1 Load libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import special
from scipy.optimize import least_squares

pio.renderers.default = 'iframe' # set iframe renderer to render plotly charts

# 2 Data Cleaning

## 2.1 Load & Save Raw Data

In [2]:
# Global car sales extracted manually from Internation Energy Agency (IEA) website
car_sales = {'year':[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022],
             'value': [66700000,71200000,73800000,77500000,80000000,81800000,86400000,86300000,85600000,81300000,71800000,74900000,74800000]}

# Convert dictionary into Pandas Dataframe
car_sales_df = pd.DataFrame(car_sales)

# Save raw data as .csv
car_sales_df.to_csv('../data/raw_data/car_sales_raw.csv',index = False, sep=",")

# Show data
car_sales_df.head()

,year,value
0,2010,66700000
1,2011,71200000
2,2012,73800000
3,2013,77500000
4,2014,80000000


In [3]:
# IEA Global EV Data Explorer API url
api_url = 'https://api.iea.org/evs/?parameter=EV%20sales&mode=Cars&category=Historical&csv=true'

# Download data from IEA website
ev_sales_df = pd.read_csv(api_url)

# Save raw data as .csv
ev_sales_df.to_csv('../data/raw_data/ev_sales_raw.csv',index = False, sep=",")

# Show data & shape
display(ev_sales_df.head())
display(ev_sales_df.shape)

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49
1,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170
2,Australia,Historical,EV sales,Cars,PHEV,2012,Vehicles,80
3,Australia,Historical,EV sales,Cars,PHEV,2013,Vehicles,100
4,Australia,Historical,EV sales,Cars,BEV,2013,Vehicles,190


(834, 8)

## 2.2 Explore Data
In this initial data exploration we will explore the ev_sales dataset:
+ study which columns will be useful
+ check for any missing data points
  
car_sales has been extracted manually. There is one value for each year and no missing year values.

In [4]:
# Show values por each column of ev_sales_df
for column in ev_sales_df.columns:
    display(ev_sales_df[column].value_counts())

region
World                26
Belgium              26
United Kingdom       26
China                26
EU27                 26
Europe               26
Germany              25
USA                  25
Spain                25
Poland               25
New Zealand          25
Netherlands          25
Japan                25
France               25
Norway               24
Canada               24
Switzerland          24
Sweden               24
Denmark              24
Other Europe         23
Rest of the world    23
Portugal             23
Australia            23
Finland              23
Italy                23
Israel               23
Iceland              22
Korea                21
Chile                21
Brazil               21
Mexico               20
Turkiye              20
South Africa         19
Austria              18
Greece               18
India                17
Name: count, dtype: int64

category
Historical    834
Name: count, dtype: int64

parameter
EV sales    834
Name: count, dtype: int64

mode
Cars    834
Name: count, dtype: int64

powertrain
BEV     443
PHEV    391
Name: count, dtype: int64

year
2019    72
2020    72
2021    72
2022    72
2015    71
2016    71
2017    71
2018    71
2014    66
2013    63
2012    59
2011    46
2010    28
Name: count, dtype: int64

unit
Vehicles    834
Name: count, dtype: int64

value
1200       12
16000      11
15000      11
11000      10
1500       10
           ..
88000       1
870000      1
510000      1
540000      1
2900000     1
Name: count, Length: 328, dtype: int64

+ In the initial data overview we have found that:
    + 'powertrain' divides ev_sales into PHEV (Plug-in Hybrid Vehicles) and BEV (Battery Electric Vehicles). Since we are only interested in BEV, we will **filter PHEV rows out** and **drop 'powertrain'** column
    + **'category', 'parameter', 'mode', and 'unit'** columns only have 1 possible value. Therefore those columns will be **dropped**.

    + 'region' and 'year' have different number of rows. This indicates we have some missing data. This can be due to:
        + years with 0 sales are excluded (year starts when there are sales), which is in agreement with the 'value' range seen
        + there are missing values for some years


## 2.3 Filter & Clean Data
+ We will **apply the actions in bold stated in the section above** and continue to study the years for each region to decide on next actions.

In [5]:
# Create a dataframe copy to work this section
ev_sales_df1 = ev_sales_df.copy()

# Filter BEV to discard PHEV
ev_sales_df1 = ev_sales_df1[ev_sales_df1['powertrain'] == 'BEV']

# Drop all columns with only 1 value
ev_sales_df1.drop(['category','parameter','mode','powertrain','unit'], axis=1, inplace=True)

# Check top 5 rows
display(ev_sales_df1.head())

,region,year,value
0,Australia,2011,49
1,Australia,2012,170
4,Australia,2013,190
5,Australia,2014,370
8,Australia,2015,760


In the *.value_counts()* above we have seen that there are no 0's in the 'value' column. This suggests that the missing years are the ones where there where no sales. Furthermore, the *.value_counts()* for the years agrees with this statement: earlier years have less values than the most recent ones. However, this doesn't 100% confirm there are no missing rows (years) for all countries. To confirm this, we will have to **check year continuity from the last year (2022) back**.

In [6]:
# Check for continuity in the years for each region
ev_sales_df1 = ev_sales_df1.sort_values(['region', 'year']) # sort values by region and year to ensure chronological order
missing = []
for region, group in ev_sales_df1.groupby('region'): # iterate df by region groups
    diffs = group['year'].diff() # calculate 
    if not diffs.isin([1, None]).all():  # if there's a difference greater than 1 year
        years = group['year'][diffs > 1]
        for year in years:
            missing.append((region, year - 1))  # previous year is missing
missing

[('Turkiye', 2013)]

The only missing value appears to be for region Turkiye, year 2013.

In [7]:
# Check Turkiye's missing value group
ev_sales_df1[ev_sales_df1['region'] == 'Turkiye']

,region,year,value
737,Turkiye,2012,92
740,Turkiye,2014,46
741,Turkiye,2015,120
744,Turkiye,2016,44
745,Turkiye,2017,77
748,Turkiye,2018,190
749,Turkiye,2019,230
752,Turkiye,2020,890
754,Turkiye,2021,3000
755,Turkiye,2022,7000


Sales around year 2013 in Turkiye were really low. In 2012, 92 cars were sold and in 2014, 46 cars were sold. Therefore we conclude the actual sales in Turkiye for the year 2013 were actually 0 - there are no missing values.

In [8]:
# Sort by region and year to ensure chronological order and reset index of dataframe
ev_sales_df1 = ev_sales_df1.sort_values(['region', 'year']).reset_index(drop=True)

## 2.4 Merge Datasets
+ We will now  combine the two datasets: **car_sales_df** and **ev_sales_df1**. To do this we will:
+ **Rename** the column **'value'** in the ev_sales_df to **'ev_sales'**
+ **Add a calculated column** to the ev_sales_df1 called **ev_sales_share**, calculated as: *ev_sales_share = ev_sales / car_sales.value*
+ **Add a new calculated column** to the ev_sales_df1 **'ev_sales_share_change'** to track the share change year by year. *'ev_sales_share_change' = 'ev_sales_share'(current year) - 'ev_sales_share'(previous year)

In [9]:
# Create a copy of the ev_sales_df for this section
ev_sales_df2 = ev_sales_df1.copy()

# Rename the column 'value' to 'ev_sales'
ev_sales_df2.rename( columns = {'value':'ev_sales'}, inplace = True)

# Add calculated column: ev_sales_share = ev_sales / car_sales.value
ev_sales_df2 = ev_sales_df2.join(car_sales_df.set_index('year'), on='year'
                                ).rename(columns={'value':'global_car_sales'}) # Join the global car_sales data

# Add calculated column: ice_sales = global_car_sales - ev_sales
ev_sales_df2['ice_sales'] = ev_sales_df2['global_car_sales'] - ev_sales_df2['ev_sales']

# Add calculated column: ev_sales_share = (ev_sales / global_car_sales) * 100
ev_sales_df2['ev_sales_share'] = ((ev_sales_df2['ev_sales'] / ev_sales_df2['global_car_sales']) * 100).round(5)

# Create an empty df too append groups
merged_df = pd.DataFrame()

for country, group in ev_sales_df2.groupby('region'):
    # Add calculated column: 'ev_sales_share_change' = 'ev_sales_share'(current year) - 'ev_sales_share'(previous year)
    group['ev_sales_share_change'] = group['ev_sales_share'] - group['ev_sales_share'].shift(+1)
    group['ev_sales_share_change'].fillna(0, inplace=True)
    # Add calculated column: 'ev_sales_share_growth'
    group['ev_sales_share_growth'] = (((group['ev_sales_share'] - group['ev_sales_share'].shift(+1)) / group['ev_sales_share'].shift(+1)) * 100).round(1)
    group['ev_sales_share_growth'].fillna(0, inplace=True)
    # Add calculated column: 'ev_sales_growth'
    group['ev_sales_growth'] = (((group['ev_sales'] - group['ev_sales'].shift(+1))/group['ev_sales'].shift(+1)) * 100).round(1)
    group['ev_sales_growth'].fillna(0, inplace=True)
    # Concatenate the group with the merged dataframe
    merged_df = pd.concat([merged_df, group])

## 2.5 Save Clean Data

In [10]:
# Save clean data as .csv
merged_df.to_csv('../data/clean_data/ev_sales_clean.csv',index = False, sep=",")
df = merged_df.copy() # Create a copy to work with in the rest of the notebook

In [11]:
df.head()

,region,year,ev_sales,global_car_sales,ice_sales,ev_sales_share,ev_sales_share_change,ev_sales_share_growth,ev_sales_growth
0,Australia,2011,49,71200000,71199951,0.00007,0.00000,0.0,0.0
1,Australia,2012,170,73800000,73799830,0.00023,0.00016,228.6,246.9
2,Australia,2013,190,77500000,77499810,0.00025,0.00002,8.7,11.8
3,Australia,2014,370,80000000,79999630,0.00046,0.00021,84.0,94.7
4,Australia,2015,760,81800000,81799240,0.00093,0.00047,102.2,105.4


# 3 Question #1 - Current Stage of EV Adoption

According to Everett M. Roger's *Diffusion of Innovation Theory*, when adopting any innovation, population is divided into the following segments:
<div align="center">
<img src="https://i0.wp.com/ondigitalmarketing.com/wp-content/uploads/2012/01/640px-Diffusionofideas.png?ssl=1" alt="Adoption curve" width=350">
</div>

To find the current stage, we must calculate the acumulated percentage for each segment:
Population segment|Accumulated percentage
------------------|-----------------------
Innovators|0-2.5%
Early Adopters|2.5-16%
Early Majority|16-50%
Late Majority|50-84%
Laggards|84-100%

We will create a function to calculate this dinamically in case we integrate this into a real data system.

In [12]:
def get_population_segment(accum_market_share: float) -> str:
    '''
    This function takes the market share and returns the population segment
    according to the Diffusion of Innovations Theory.

    Input:
    accum_market_share: market share as percent i.e 15 for 15%

    Output:
    population:_segment_name: segment name i.e Innovators
    
    '''
    if accum_market_share < 2.5:
        return 'Innovators'
    elif accum_market_share < 16:
        return 'Early Adopters'
    elif accum_market_share < 50:
        return 'Early Majority'
    elif accum_market_share < 84:
        return 'Late Majority'
    elif accum_market_share <= 100:
        return 'Laggards'

Next, we simply have to filter the region to 'World', get the last value, and use the function to return the current stage.

In [13]:
# Filter df to get current_ev_sales_share
current_ev_sales_share = df[df['region'] == 'World']['ev_sales_share'].tail(1).iloc[0]

# Print answer to question #2
print('Answer to question #1: We are currently in the {} stage with {}% of car sales being electric.'.format(get_population_segment(current_ev_sales_share),current_ev_sales_share.round(2)))

Answer to question #1: We are currently in the Early Adopters stage with 9.76% of car sales being electric.


# 4 Question #2 - Leading countries in EV sales

In [14]:
# Create function that returns a ranked table of countries by ev_sales and a fig to plot data in a pie chart
def create_country_sales_rank(df: pd.DataFrame, current_year: int) -> tuple:
    '''
    This function creates a ranked table sorted by 'ev_sales' and 
    a fig to plot a pie chart from a df.

    Input:
    df: pandas DataFrame

    Output:
    tuple: ranked_df, fig
    
    '''

    # List regions that are not countries to filter them out
    exclude_regions = ['World','Europe','EU27','Other Europe']
    
    # Filter df to get rank of countries
    rank_df = df[(df['year']==current_year) & (~df['region'].isin(exclude_regions))][['region','ev_sales']].sort_values('ev_sales', ascending = False)
    
    # Create rank column
    rank_df['Rank'] = rank_df.reset_index().index + 1

    # Rename columns to show in table or plot
    rank_df.rename(columns={'region':'Country','ev_sales':'Sales','ev_sales_country_share':'Sales share (%)'}, inplace=True)

    # Filter out countries with sales < 80000 to represent only large countries
    rank_df.loc[rank_df['Sales'] < 80000, 'Country'] = 'All other countries' 

    # Create pie chart fig
    fig = px.pie(rank_df, values='Sales', names='Country', title='Sales by Country')
    
    return rank_df[['Rank', 'Country', 'Sales']].head(10), fig
    
# Apply function
sales_rank_df, fig = create_country_sales_rank(df, 2022)

# Plot pie chart
fig.show()

# Display ranked table
display(sales_rank_df)

,Rank,Country,Sales
82,1,China,4400000
416,2,USA,800000
158,3,Germany,470000
429,4,United Kingdom,270000
145,5,France,210000
294,6,Norway,150000
243,7,Korea,120000
381,8,Sweden,96000
57,9,Canada,85000
268,10,All other countries,73000


# 5 Question #3 - Leading countries in EV sales growth

In [15]:
# Create function that returns a ranked table of countries by ev_sales_share_growth
def create_country_sales_growth_rank_df(df: pd.DataFrame, current_year: int) -> pd.DataFrame:
    '''
    This function creates a ranked table sorted by 'ev_sales_share_growth' and from a df.

    Input:
    df: pandas DataFrame

    Output:
    df: pandas DataFrame
    
    '''
    # Define the ranking column
    ranking_column = 'ev_sales_share_growth'
    
    # List regions that are not countries to filter them out
    exclude_regions = ['World','Europe','EU27','Other Europe','Rest of the world']
    
    # Filter df to get rank of countries with at least sales > 10000
    rank_df = df[(df['year']== current_year) & (~df['region'].isin(exclude_regions)) & (df['ev_sales']> 10000)].sort_values(ranking_column, ascending = False)
    
    # Create rank column
    rank_df['Rank'] = rank_df.reset_index().index + 1

    # Rename columns to show in table
    rank_df.rename(columns={'region':'Country','ev_sales':'Sales','ev_sales_share_growth':'Sales share growth (%)'}, inplace=True)

    return rank_df[['Rank', 'Country', 'Sales share growth (%)', 'Sales']].head(10)

create_country_sales_growth_rank_df(df, 2022)

,Rank,Country,Sales share growth (%),Sales
192,1,India,300.6,48000
281,2,New Zealand,179.7,19000
230,3,Japan,177.7,61000
204,4,Israel,145.7,27000
320,5,Poland,94.8,14000
11,6,Australia,94.4,33000
416,7,USA,70.4,800000
381,8,Sweden,68.6,96000
243,9,Korea,66.9,120000
34,10,Belgium,65.4,38000


# 6 Question #4 - World EV adoption trend

In [16]:
def create_plot_and_fit(df: pd.DataFrame) -> tuple:
    '''
    Create a plot and fit different models to the data.

    Args:
        df (pd.DataFrame): DataFrame containing the data with columns 'year' and 'ev_sales_share'.

    Returns:
        tuple: A tuple containing the Plotly figure object and a DataFrame with R-squared values.

    '''

    # Assign x and y data
    x_data = df['year']
    y_data = df['ev_sales_share']
    
    # Define the R-squared function
    def r_squared(y, y_pred):
        residual = np.sum((y - y_pred) ** 2)
        total = np.sum((y - np.mean(y)) ** 2)
        r_sq = 1 - (residual / total)
        return r_sq
    
    # Fit a linear model to the data
    def linear_func(x, a, b):
        return a * x + b
        
    linear_params, _ = curve_fit(linear_func, x_data, y_data)
    
    # Fit a quadratic model to the data
    def quadratic_func(x, a, b, c):
        return a * x**2 + b * x + c
    
    quadratic_guesses = [1, 1, 1]  # Initial guesses for the quadratic fit
    quadratic_params, _ = curve_fit(quadratic_func, x_data, y_data, p0=quadratic_guesses)
    
    # Fit an exponential model to the data
    def exponential_func(x, a, b, c):
        return a * np.exp(b * (x - 2010)) + c
    
    exponential_guesses = [1, 1, 1]  # Initial guesses for the exponential fit
    exponential_params, _ = curve_fit(exponential_func, x_data, y_data, p0=exponential_guesses)
    
    # Calculate R-squared values
    r_squared_values = {
        'Fit': ['Linear', 'Quadratic', 'Exponential'],
        'R-Squared': [r_squared(y_data, linear_func(x_data, *linear_params)), 
                      r_squared(y_data, quadratic_func(x_data, *quadratic_params)), 
                      r_squared(y_data, exponential_func(x_data, *exponential_params))]}
    
    # Create a DataFrame with R-squared values
    r_squared_values_df = pd.DataFrame(r_squared_values)
    
    # Visualize the data and fits using Plotly
    fig = go.Figure()
    
    # Original data points
    fig.add_trace(go.Bar(x=x_data, y=y_data, name='Electric Vehicles', marker_color='#90EE90'))
    
    # Linear fit
    linear_fit = linear_func(x_data, *linear_params)
    fig.add_trace(go.Scatter(x=x_data, y=linear_fit, mode='lines', name='Linear Fit', line=dict(color='#000080', dash='dashdot')))
    
    # Quadratic fit
    quadratic_fit = quadratic_func(x_data, *quadratic_params)
    fig.add_trace(go.Scatter(x=x_data, y=quadratic_fit, mode='lines', name='Quadratic Fit', line=dict(color='#FFA500', dash='dash')))
    
    # Exponential fit
    exponential_fit = exponential_func(x_data, *exponential_params)
    fig.add_trace(go.Scatter(x=x_data, y=exponential_fit, mode='lines', name='Exponential Fit', line=dict(color='#FF0000', width=2)))

    # Set layout configuration
    fig.update_layout(
        title="EV Sales Share Trend Line Fit",
        xaxis_title="",
        yaxis_title="Sales Share (%)",
        xaxis={'type': 'category'},
        legend=dict(
            orientation='h',
            yanchor='top',
            y=-0.2,
            xanchor='center',
            x=0.5
        ),
    )

    return fig, r_squared_values_df

In [17]:
# Filtar world data
world_df = df[df['region'] == 'World']

# Use the function
fig, r_sq_df = create_plot_and_fit(world_df)

# Display R-square metrics
display(r_sq_df)

# Plot the chart
fig.show()

,Fit,R-Squared
0,Linear,0.630410
1,Quadratic,0.903264
2,Exponential,0.992409


# 7 Question #5 - Finding the EV curve of adoption

In [18]:
# Create a function the fits a normal distribution curve to the data points provided. Returns mean and sd.
def find_normal_curve(df: pd.DataFrame) -> tuple:
    '''
    Find the parameters (mean and standard deviation) of a normal distribution curve that best fits the given data points.

    Args:
        x_values (pd.Series): Series of x values.
        y_values (pd.Series): Series of y values.

    Returns:
        tuple: A tuple containing the mean and standard deviation of the fitted normal curve.

    '''
    
    # Filter world data
    world_df = df[df['region'] == 'World']

    # Get x, y values
    x_values = world_df['year']
    y_values = world_df['ev_sales_share'] / 100

    # Define the system of equations
    def equations(p):
        mu, sd = p
        return y_values - 0.5 * (1 + special.erf((x_values - mu) / (sd * np.sqrt(2))))
    
    # Initial guess for the mean and standard deviation
    mu_init, sd_init = 2028, 2
    
    # Solve the system of equations
    solution = least_squares(equations, (mu_init, sd_init))
    
    mu, sd = solution.x
    
    return mu, sd

In [19]:
mu, sd = find_normal_curve(df)
print('Mean (μ):', mu)
print('Standard Deviation (σ):', sd)

Mean (μ): 2027.1603015453518
Standard Deviation (σ): 3.9613199624616042


In [20]:
def get_year_from_cdf(mu: float, sd: float, cdf: float):
    '''
    Get the year corresponding to a given cumulative distribution function (CDF) value of a normal distribution.

    Args:
        mu (float): Mean of the normal distribution.
        sd (float): Standard deviation of the normal distribution.
        cdf (float): Cumulative distribution function (CDF) value.

    Returns:
        float: The year corresponding to the given CDF value.

    '''
    rv = norm(loc=mu, scale=sd)
    return rv.ppf(cdf)

get_year_from_cdf(mu,sd,0.9)

2032.2369373448676

In [21]:
# Test calculator: Calculate the CDF and PDF for a specific value
x = 2027

# Create an instance of the normal distribution
rv = norm(loc=mu, scale=sd)

cdf_value = rv.cdf(x)
pdf_value = rv.pdf(x)

print("CDF:", cdf_value)
print("PDF:", pdf_value)

CDF: 0.4838605275356998
PDF: 0.10062700643883253


This project we will analyze the increasing demand of EVs and who's leading it, figure out in which stage of adoption we are and try to estimate how long will it take for the all car transportation to become electric by finding the EV adoption curve.

In [22]:
def get_current_cdf_and_delta(df) -> tuple:
    '''
    Get the current cumulative distribution function (CDF) and its delta from a DataFrame.

    Args:
        df (pd.DataFrame): DataFrame containing the data.

    Returns:
        tuple: A tuple containing the current year, current CDF, and delta.

    '''
    
    # Get current CDF
    current_cdf = (df[df['region'] == 'World']['ev_sales_share'].tail(2).iloc[1]).round(3)
    
    # Get current CDF delta
    delta = df[df['region'] == 'World']['ev_sales_share'].tail(2).iloc[0].round(3)
    
    # Get current year
    current_year = df[df['region'] == 'World']['year'].tail(1).iloc[0]

    return current_year, current_cdf, delta

# Use function
current_year, current_adoption, delta = get_current_cdf_and_delta(df)

# Print results
print(current_year, current_adoption, delta)

2022 9.759 6.142


In [23]:
def create_adoption_curve(mu: float, sd: float, current_cdf: float):
    '''
    Create an adoption curve plot showing the cumulative distribution function (CDF) and probability density function (PDF) of a normal distribution.

    Args:
        mu (float): Mean of the normal distribution.
        sd (float): Standard deviation of the normal distribution.
        current_adoption (float): Current adoption rate (between 0 and 1).

    Returns:
        go.Figure: Plotly figure object representing the adoption curve.

    '''

    # Generate the x values with a wider range
    x = np.linspace(mu - 3 * sd, mu + 3 * sd, 1000)
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Calculate the CDF using the same x values
    cdf_y = norm.cdf(x, mu, sd)
    
    # Plot the CDF
    fig.add_trace(go.Scatter(x=x, y=cdf_y, mode='lines', name='CDF', line=dict(color='orange'), yaxis='y2'),secondary_y=True)
    
    # Plot the normal distribution using a separate y-axis
    fig.add_trace(go.Scatter(x=x, y=norm.pdf(x, mu, sd), mode='lines', name='PDF', line=dict(color='darkblue'), yaxis='y2'),secondary_y=False)
    
    # Shade the left tail up to current_adoption
    x_fill = np.linspace(mu - 3 * sd, norm.ppf(current_cdf, mu, sd), 100)
    fig.add_trace(go.Scatter(x=x_fill, y=norm.pdf(x_fill, mu, sd), fill='tozeroy', fillcolor='#98FB98', opacity=0, name='Current PDF', yaxis='y2'),secondary_y=False)
    
    # Add a vertical line on the current_adoption
    x_line = norm.ppf(current_cdf, mu, sd)
    vertical_line2 = go.Scatter(x=[x_line, x_line], y=[0, norm.pdf(x_line, mu, sd)], mode='lines', name='Vertical Line 2', line=dict(color='darkblue', width=2), yaxis='y2')
    fig.add_trace(vertical_line2, secondary_y=False)
    
    # Add annotation with the current adoption
    fig.add_annotation(x=x_line, y=norm.pdf(x_line, mu, sd)+0.0115, text='<b>Current CDF</b>', showarrow=False, yshift=10, font=dict(size=12, color='darkgreen'), yref='y1')
    fig.add_annotation(x=x_line, y=norm.pdf(x_line, mu, sd)+0.006, text='<b>'+str(current_cdf*100)+'%</b>', showarrow=False, yshift=10, font=dict(size=12, color='darkgreen'), yref='y1')
    
    # Segment CDF boundaries
    segments = [2.5, 16, 50, 84]
    
    for segment in segments:
        # Add a vertical line for every segment
        x_line = norm.ppf(segment/100, mu, sd)
        vertical_line2 = go.Scatter(x=[x_line, x_line], y=[0, norm.pdf(x_line, mu, sd)], mode='lines', name='Vertical Line 2', line=dict(color='darkblue', width=2), yaxis='y2')
        fig.add_trace(vertical_line2, secondary_y=False)
    
    annotations = {'Innovators':[-1.7, 0.14, ''],
                   '':[-1.95, 0.14, '2.5%'],
                   'Early Adopters':[-1, 0.5, '16%'],
                   'Early Majority':[0, 0.78, '50%'],
                   'Late Majority':[1, 0.5, '84%'],
                   'Laggards':[2, 0.025, ''],}
    
    for key, value in annotations.items():
        fig.add_annotation(x=mu+(value[0]*sd)-2, y=0.01, text='<b>'+key+'</b>', showarrow=False, yshift=10, font=dict(size=12), yref='y2')
        fig.add_annotation(x=mu+(value[0]*sd), y=value[1], text=value[2], showarrow=False, yshift=10, font=dict(size=12), yref='y2')
    
    # Customize the layout
    fig.update_layout(
        xaxis=dict(showgrid=True, zeroline=False, dtick=1),  # Show gridlines, zeroline, and set dtick to 1 for 1 unit interval in x-axis
        yaxis=dict(showgrid=False, zeroline=True, range=[0, 0.13], side='left', showticklabels=False ),
        yaxis2=dict(showgrid=True, zeroline=False, range=[0, 1], side='right', overlaying='y', ticktext=['0%', '25%', '50%', '75%', '100%'], tickvals=[0, 0.25, 0.5, 0.75, 1]),# 0.195]),  # Create a secondary y-axis without tick labels
        showlegend=False,
        margin=dict(l=2, r=2, t=2, b=20)
    )
    return fig


mu, sd = 2027.16, 3.96
current_adoption = 0.0978

fig = create_adoption_curve(mu, sd, current_adoption)

# Display the figure
fig.show()


# Extra Application Plots

## Electric vs. Combustion Sales

In [24]:
# Create a function to plot a stacked bar chart of Electric vs. Combustion sales
def create_stacked_bar_chart(df: pd.DataFrame):
    '''
    Create a stacked bar chart showing global car sales split by combustion/EV over time.

    Args:
        df (pd.DataFrame): DataFrame containing the data.

    Returns:
        fig: Plotly figure object representing the stacked bar chart.

    '''
    
    # Filter & rename dataframe
    world_df = df[df['region'] == 'World'].rename(columns={'ev_sales': 'Electric', 'ice_sales': 'Combustion'})
    
    # Create the stacked bar chart
    fig = px.bar(world_df, x='year', y=['Electric', 'Combustion'], title='Global Car Sales',
                color_discrete_sequence=['#90EE90', '#ADD8E6'])
    
    # Update the layout
    fig.update_layout(
        barmode='stack',
        legend=dict(
            orientation='h',
            yanchor='top',
            y=-0.2,
            xanchor='center',
            x=0.5
        ),
        xaxis_title=None,
        yaxis_title='Sales',
        xaxis={'type': 'category'})

    return fig

# Use the function to get the fig
fig = create_stacked_bar_chart(df)

# Display the chart
fig.show()

## World Sales Share with Trendline

In [25]:
# Create a function to plot the world sales share with an exponential trendline
def plot_world_sales_share(df: pd.DataFrame):
    '''
    Plot the world electric vehicle (EV) sales share over time and an exponential fit.

    Args:
        df (pd.DataFrame): DataFrame containing the data.

    Returns:
        fig: Plotly figure object representing the plot.

    '''

    # Filter world data
    world_df = df[df['region'] == 'World']

    # Define exponential function
    def exponential_func(x, a, b, c):
        return a * np.exp(b * (x - 2010)) + c
    
    # World EV Sales
    fig = px.bar(world_df, x='year', y='ev_sales_share', title='World EV Sales')

    # Configure plot settings
    fig.update_layout(
        xaxis_title='',
        yaxis_title='World car sales share (%)',
        xaxis={'type': 'category'},
        showlegend=False
    )
    
    # Add exponential line
    x_values = world_df['year']
    a, b, c = 0.01396663, 0.54538719, 0.11010673
    y_exp = exponential_func(x_values, a, b, c)

    fig.add_trace(go.Scatter(x=x_values, y=y_exp, mode='lines', name='Exponential Fit'))

    return fig

fig = plot_world_sales_share(df)
fig.show()

## Sales Share Trend Line by region

In [26]:
# Create function
def create_plot_and_fit(df: pd.DataFrame) -> tuple:
    '''
    Create a plot showing the car sales share over time and fit different models to the data.

    Args:
        df (pd.DataFrame): DataFrame containing the data.

    Returns:
        tuple: A tuple containing the Plotly figure object representing the plot and a DataFrame with R-squared values for each fit.

    '''

    # Assign x and y data
    x_data = df['year']
    y_data = df['ev_sales_share']
    
    # Define the R-squared function
    def r_squared(y, y_pred):
        residual = np.sum((y - y_pred) ** 2)
        total = np.sum((y - np.mean(y)) ** 2)
        r_sq = 1 - (residual / total)
        return r_sq
    
    # Fit a linear model to the data
    def linear_func(x, a, b):
        return a * x + b
    
    linear_params, _ = curve_fit(linear_func, x_data, y_data)
    
    # Fit a quadratic model to the data
    def quadratic_func(x, a, b, c):
        return a * x**2 + b * x + c
    
    quadratic_guesses = [1, 1, 1]  # Initial guesses for the quadratic fit
    quadratic_params, _ = curve_fit(quadratic_func, x_data, y_data, p0=quadratic_guesses)
    
    # Fit an exponential model to the data
    def exponential_func(x, a, b, c):
        return a * np.exp(b * (x - 2010)) + c
    
    exponential_guesses = [1, 1, 1]  # Initial guesses for the exponential fit
    exponential_params, _ = curve_fit(exponential_func, x_data, y_data, p0=exponential_guesses)
    
    # Calculate R-squared values
    r_squared_values = {
        'Linear': r_squared(y_data, linear_func(x_data, *linear_params)),
        'Quadratic': r_squared(y_data, quadratic_func(x_data, *quadratic_params)),
        'Exponential': r_squared(y_data, exponential_func(x_data, *exponential_params))
    }
    
    # Create a DataFrame with R-squared values
    r_squared_values_df = pd.DataFrame.from_dict(r_squared_values, orient='index', columns=['R-squared'])
    
    # Visualize the data and fits using Plotly
    fig = go.Figure()
    
    # Original data points
    fig.add_trace(go.Bar(x=x_data, y=y_data, name='Electric', marker_color='#90EE90'))
    
    # Linear fit
    linear_fit = linear_func(x_data, *linear_params)
    fig.add_trace(go.Scatter(x=x_data, y=linear_fit, mode='lines', name='Linear Fit', line=dict(color='#f39c12', dash='dashdot')))
    
    # Quadratic fit
    quadratic_fit = quadratic_func(x_data, *quadratic_params)
    fig.add_trace(go.Scatter(x=x_data, y=quadratic_fit, mode='lines', name='Quadratic Fit', line=dict(color='#27ae60', dash='dash')))
    
    # Exponential fit
    exponential_fit = exponential_func(x_data, *exponential_params)
    fig.add_trace(go.Scatter(x=x_data, y=exponential_fit, mode='lines', name='Exponential Fit', line=dict(color='#e74c3c', width=3)))
    
    fig.update_layout(
        title="Car Sales Share",
        xaxis_title="",
        yaxis_title="Sales Share (%)",
        xaxis={'type': 'category'}
    )

    return fig, r_squared_values_df


df1 = df[df['region'] == 'China']

fig , r_squared_values_df = create_plot_and_fit(df1)

display(r_squared_values_df)
fig.show()

,R-squared
Linear,0.602054
Quadratic,0.878371
Exponential,0.983669
